In [ ]:
from os import environ
if environ.get('netlogo_path') is not None:
    netlogo_path = environ.get('netlogo_path')
else:
    netlogo_path = "C:/Program Files/NetLogo 6.0.2/"
import nl4py
# Replace argument to startServer(...) with the location of your NetLogo installation 
nl4py.startServer(netlogo_path)
import time
def simulate(workspace_):
    workspace_.command("stop")
    # Same netlogo commands as used for the PyNetLogo evaluation
    workspace_.command("random-seed 999")
    workspace_.command("set initial-number-sheep 150")
    workspace_.command("set initial-number-wolves 150")
    workspace_.command("set grass-regrowth-time 50")
    workspace_.command("set sheep-gain-from-food 25")
    workspace_.command("set wolf-gain-from-food 50")
    workspace_.command("set sheep-reproduce 10")
    workspace_.command("set wolf-reproduce 10")
    workspace_.command("set show-energy? false")
    workspace_.command('set model-version "sheep-wolves-grass"')
    workspace_.command('setup')
    workspace_.scheduleReportersAndRun(["ticks",'count sheep','count wolves'], 0,1,100,"go")    

In [ ]:
def measureExecutionTime(runsNeeded):
    startTime = int(round(time.time() * 1000))    
    runsDone = 0
    runsStarted = 0
    allResults = []
    # Make sure we start 8 headless workspaces to compare to 8 IPCluster engines running PyNetLogo
    try:
        for i in range(0,8):
            workspace = nl4py.newNetLogoHeadlessWorkspace()
            workspace.openModel('../models/Wolf Sheep Predation.nlogo')
            simulate(workspace)
            runsStarted = runsStarted + 1
        while (runsDone < runsNeeded):
            for workspace in nl4py.getAllHeadlessWorkspaces():
                newResults = workspace.getScheduledReporterResults()
                # Make sure the models have finished running.
                if len(newResults) > 0:
                    allResults.extend(newResults)
                    runsDone = runsDone + 1
                    if runsStarted < runsNeeded:
                        simulate(workspace)
                        runsStarted = runsStarted + 1
    except Exception as e:
        print(e)
        pass
    stopTime = int(round(time.time() * 1000))
    executionTime = stopTime - startTime
    return executionTime

In [ ]:
import os
outputFile = '../output/5.3_output.csv'
out = open (outputFile, "a+")

In [ ]:
for runs in [5000, 10000, 15000]:
    executionTime = measureExecutionTime(runs)
    print("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime))
    out.write("nl4py,scheduledReporters," + str(runs) + "," + str(executionTime) + "\n")
    nl4py.deleteAllHeadlessWorkspaces()
    out.flush()

In [ ]:
out.close()
nl4py.stopServer()